In [1]:
%load_ext autoreload

In [2]:
%autoreload 2
from datetime import datetime
from matplotlib import colors
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from neural_analysis.matIO import loadmat
from neural_analysis import spectra
import numpy as np
import os
import pandas as pd
import re
import scipy
from statsmodels.tsa.api import VAR
import sys
import time
from tqdm.auto import tqdm
import traceback as tb

sys.path.append('../../..')
from multiproc_VAR_p_grid_search import predict_VAR_p
from plot import plot_individual_areas, plot_multipopulation
from up_down import get_up_down
from utils import compile_folder, compute_summary_statistics, get_data_class, get_result_path, load, load_session_data, run_window_selection, save

# Doing Stuff

In [111]:
# results_folder = 'TEST_Mary-Anesthesia-20160809-01_num_window_samples_1_Jun-12-2022_1138' # TEST: lag 4, 1 sample
# results_folder = 'BADRUN_Mary-Anesthesia-20160809-01_num_window_samples_5_Jun-12-2022_1141' # lag 15, 5 samples
# results_folder = 'BETTER_Mary-Anesthesia-20160809-01_num_window_samples_2_Jun-19-2022_0249'
# results_folder = 'AHA_Mary-Anesthesia-20160809-01_num_window_samples_2_Jun-19-2022_1431'
# results_folder = 'Mary-Anesthesia-20160809-01_num_window_samples_2_Jun-19-2022_1957'
# results_dir = f'/om/user/eisenaj/ChaoticConsciousness/results/propofolPuffTone/VAR_p/grid_search/{results_folder}'

In [119]:
results_dir = f'/om/user/eisenaj/ChaoticConsciousness/results/propofolPuffTone/VAR_p/grid_search'

In [125]:
files_to_process = []
for filename in os.listdir(results_dir):
    if not os.path.exists(os.path.join(results_dir, filename, '_analysis')):
        files_to_process.append(filename)

In [127]:
def pick_2d_optimum(mat, thresh=0.95):
    true_min = mat.min()
    i_vals, j_vals = np.where(mat*thresh - true_min <= 0)
    selected_i = np.min(i_vals)
    selected_j = np.min(j_vals[i_vals == selected_i])
    selected_i, selected_j

    return selected_i, selected_j

In [124]:
for filename in files_to_process:
    print("-----------------")
    print(filename)
    print("-----------------")
    session = filename.split('_')[1]
    
    thresh = 0.9
    selected_model_parameters = {}
    test_mse_mats = {}
    grid_search_dfs = {}
    iterator = tqdm(total = 5*4)
    for area in ['vlPFC', 'FEF', '7b', 'CPB', 'all']:
        selected_model_parameters[area] = {}
        test_mse_mats[area] = {}
        grid_search_dfs[area] = {}
        for section in ['pre', 'induction', 'during', 'post']:
            grid_search_df = load(os.path.join(results_dir, filename, area, section))
            grid_search_dfs[area][section] = grid_search_df

            windows = np.sort(grid_search_df.window.unique())
            lags = np.sort(grid_search_df.p.unique())
            max_lag = lags.max()

            test_mse_mat = np.zeros((len(windows), len(lags)))
            for i, window in enumerate(windows):
                for j, p in enumerate(lags):
                    test_mse_mat[i, j] = grid_search_df[np.logical_and(grid_search_df.window == window, grid_search_df.p == p)].test_mse.mean()
            test_mse_mat[np.isnan(test_mse_mat)] = test_mse_mat[~np.isnan(test_mse_mat)].max()*1.2
            w_ind, p_ind = pick_2d_optimum(test_mse_mat, thresh)

            test_mse_mats[area][section] = test_mse_mat

            selected_model_parameters[area][section] = {'window': windows[w_ind], 'p': lags[p_ind]}
            iterator.update()
    iterator.close()
    
    analysis = dict(
        selected_model_parameters=selected_model_parameters,
        test_mse_mats=test_mse_mats,
        thresh=thresh  
    )
    
    save(analysis, os.path.join(results_dir, filename, '_analysis'))

-----------------
Mary-Anesthesia-20160809-01_num_window_samples_2_Jun-19-2022_1957
-----------------


  0%|          | 0/20 [00:00<?, ?it/s]

In [123]:
# fig, axs = plt.subplots(5, 4, figsize=(14, 25))

# for i, area in enumerate(['vlPFC', 'FEF', '7b', 'CPB', 'all']):
#     for j, section in enumerate(['pre', 'induction', 'during', 'post']):
#         ax = axs[i][j]
#         ax.pcolormesh(np.log(test_mse_mats[area][section]))
#         ax.set_xticks(np.arange(0, max_lag) + 0.5)
#         ax.set_xticklabels([lags[i] for i in np.arange(0, max_lag)])
#         ax.set_yticks(np.arange(0, len(windows)) + 0.5)
#         ax.set_yticklabels([f"{windows[i]:.2f}" for i in np.arange(0, len(windows))])
#         ax.set_xlabel('Lags')
#         ax.set_ylabel('Window Length (s)')
#         ax.set_title(f"{area} - {section}")
# #         cbar = plt.colorbar(label='Log MSE')
#         ax.scatter([selected_model_parameters[area][section]['p'] - 0.5], [int(selected_model_parameters[area][section]['window']/0.5) - 0.5], color='red', s=40, label='min')

# plt.tight_layout()
# plt.show()

In [128]:
selected_model_parameters

{'vlPFC': {'pre': {'window': 8.0, 'p': 10},
  'induction': {'window': 8.5, 'p': 10},
  'during': {'window': 8.5, 'p': 10},
  'post': {'window': 8.0, 'p': 10}},
 'FEF': {'pre': {'window': 5.5, 'p': 10},
  'induction': {'window': 7.0, 'p': 10},
  'during': {'window': 8.0, 'p': 10},
  'post': {'window': 9.0, 'p': 10}},
 '7b': {'pre': {'window': 7.0, 'p': 10},
  'induction': {'window': 7.0, 'p': 10},
  'during': {'window': 8.0, 'p': 10},
  'post': {'window': 8.0, 'p': 10}},
 'CPB': {'pre': {'window': 8.0, 'p': 10},
  'induction': {'window': 7.0, 'p': 10},
  'during': {'window': 6.0, 'p': 10},
  'post': {'window': 3.0, 'p': 10}},
 'all': {'pre': {'window': 9.5, 'p': 10},
  'induction': {'window': 9.5, 'p': 10},
  'during': {'window': 8.0, 'p': 10},
  'post': {'window': 8.0, 'p': 9}}}

In [129]:
eyes_close = session_info['eyesClose'][-1] if isinstance(session_info['eyesClose'], np.ndarray) else session_info['eyesClose'] 
section_times = dict( 
    pre=(0, session_info['drugStart'][0]),
    induction=(session_info['drugStart'][0], eyes_close),
    during=(eyes_close, session_info['drugEnd'][1]),
    post=(session_info['drugEnd'][1], T*dt)
)
sections = list(section_times.keys())

In [130]:
section_times

{'pre': (0, 1875.3599666666667),
 'induction': (1875.3599666666667, 2315.672266666667),
 'during': (2315.672266666667, 5494.063133333334),
 'post': (5494.063133333334, 7085.396)}

In [132]:
areas = ['vlPFC', 'FEF', 'CPB', '7b', 'all']

In [133]:
sections = list(selected_model_parameters['vlPFC'].keys())

In [134]:
sections

['pre', 'induction', 'during', 'post']

In [135]:
num_windows = int(np.ceil(section_times[section]/window))

TypeError: only size-1 arrays can be converted to Python scalars

In [146]:
8*686

5488

In [138]:
tasks = []
i = 0
for area in areas:
    for section in sections:
        window = selected_model_parameters[area][section]['window']
        p = selected_model_parameters[area][section]['p']
        stride = window
        window_inds = np.arange(int(section_times[section][0]/stride), int(np.ceil(section_times[section][1]/stride)))
        for window_ind in window_inds:
            tasks.append(area, section, window, stride, p, window_ind, T_pred, directory, N, dt, results_dir)

(0, 1875.3599666666667)
8.0
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234]
-----------------

In [147]:
import multiprocessing as mp

In [148]:
queue = mp.Manager().JoinableQueue()

In [150]:
queue.qsize()

0

In [152]:
for i in tqdm(range(1000000)):
    queue.put(i)

  0%|          | 0/1000000 [00:00<?, ?it/s]

In [154]:
queue.qsize()

1000000